In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import ipywidgets as widgets
from IPython.display import display, Markdown

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 6), 'figure.dpi': 130,
                     'axes.titlesize': 'x-large', 'axes.labelsize': 'large',
                     'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized for Advanced Principal-Agent Models.")

# Part 5: Microeconomic Theory
## Chapter 5.05: Principal-Agent Models: The Economics of Asymmetric Information and Incentives

### Table of Contents
1.  [The Moral Hazard Problem (Hidden Action)](#1.-The-Moral-Hazard-Problem-(Hidden-Action))
    *   [1.1 The Static Incentive-Insurance Trade-Off](#1.1-The-Static-Incentive-Insurance-Trade-Off)
    *   [1.2 The First-Order Approach (Continuous Effort)](#1.2-The-First-Order-Approach-(Continuous-Effort))
    *   [1.3 Dynamic Moral Hazard](#1.3-Dynamic-Moral-Hazard)
2.  [The Adverse Selection Problem (Hidden Information)](#2.-The-Adverse-Selection-Problem-(Hidden-Information))
    *   [2.1 The Optimal Screening Menu](#2.1-The-Optimal-Screening-Menu)
    *   [2.2 The Revelation Principle](#2.2-The-Revelation-Principle)
3.  [Application: Optimal Unemployment Insurance](#3.-Application:-Optimal-Unemployment-Insurance)
4.  [Chapter Summary](#4.-Chapter-Summary)
5.  [Exercises](#5.-Exercises)

### Introduction: The Challenge of Asymmetric Information

In many economic relationships, one party has better information than another. This **asymmetric information** is a fundamental source of market friction. The **Principal-Agent framework** is the workhorse model economists use to analyze these situations.

A principal-agent relationship exists whenever a **Principal** (e.g., an employer) hires an **Agent** (e.g., an employee) to perform a task on their behalf. The problem arises because their interests are not aligned, and the Principal cannot perfectly monitor the Agent. There are two main classes of information problems:

1.  **Moral Hazard (Hidden Action):** Arises *after* a contract is signed. The Principal cannot observe the Agent's actions (e.g., effort). The challenge is to design a contract that incentivizes the Agent to act in the Principal's interest.

2.  **Adverse Selection (Hidden Information):** Arises *before* a contract is signed. The Agent has private information about their own "type" (e.g., skill). The challenge is to design a menu of contracts that induces Agents to reveal their type through their choices.

### 1. The Moral Hazard Problem (Hidden Action)

#### 1.1 The Static Incentive-Insurance Trade-Off
In the canonical static model, a risk-neutral Principal hires a risk-averse Agent. The Principal's goal is to design a wage contract $(w_H, w_L)$ based on observable outcomes (high or low output) to induce unobservable high effort ($e_H$) at minimum cost.

The Principal faces a trade-off. To induce high effort, they must offer a wage spread ($w_H > w_L$), linking pay to performance. However, this exposes the risk-averse agent to income risk. The optimal **second-best contract** balances this **incentive-insurance trade-off**. It provides the *minimum* amount of incentive (and thus imposes the minimum amount of costly risk) needed to induce high effort.

In [ ]:
sec("Interactive: The Incentive-Insurance Trade-Off")

def solve_moral_hazard(pi_H=0.8, pi_L=0.3, c_H=1.0, u_bar=4.0, gamma=2.0):
    u_inv = lambda u, g: ((1 - g) * u)**(1 / (1 - g)) if not np.isclose(g, 1) else np.exp(u)
    delta_pi = pi_H - pi_L
    u_L = u_bar + c_H - (pi_H / delta_pi) * c_H
    u_H = u_L + c_H / delta_pi
    w_H, w_L = u_inv(u_H, gamma), u_inv(u_L, gamma)
    expected_wage = pi_H * w_H + (1 - pi_H) * w_L
    
    note(f"For Agent RRA (γ) = {gamma:.1f}:")
    print(f"  - Optimal Contract: w_H = ${w_H:.2f}, w_L = ${w_L:.2f}")
    print(f"  - Required Wage Spread (Incentive): ${w_H - w_L:.2f}")
    print(f"  - Expected Wage Cost to Principal: ${expected_wage:.2f}")

widgets.interact(solve_moral_hazard, 
                 pi_H=widgets.FloatSlider(min=0.5, max=0.95, step=0.05, value=0.8, description='Prob(High|Effort)'),
                 gamma=widgets.FloatSlider(min=0.5, max=5.0, step=0.5, value=2.0, description='Risk Aversion (γ)'));

#### 1.2 The First-Order Approach (Continuous Effort)
When effort is a continuous choice, the Principal's problem can often be simplified by replacing the agent's full incentive compatibility constraint with just its first-order condition. This **first-order approach** is valid if the agent's optimization problem is concave, which is ensured by the **Convexity of the Distribution Function Condition (CDFC)**.

This approach leads to the elegant **Mirrlees-Holmström condition**, which states that the agent's inverse marginal utility of income should be a linear function of the likelihood ratio:
$$ \frac{1}{u'(w(q))} = \lambda + \mu \frac{f_e(q|e)}{f(q|e)} $$
The term $\frac{f_e(q|e)}{f(q|e)}$ is the score of the likelihood function. It measures how informative output $q$ is about the agent's effort. The optimal contract makes pay most sensitive to outcomes that are most informative about effort.

#### 1.3 Dynamic Moral Hazard
In a multi-period relationship, the Principal has another tool: the promise of future rewards. The Principal can use the agent's **continuation value** as an incentive device. A good outcome today can be rewarded not just with a higher wage, but with a promotion or the continuation of a profitable relationship. This allows the Principal to rely less on direct, risky wage incentives in any single period, leading to better risk-sharing. This is the logic behind efficiency wages and career concerns.

### 2. The Adverse Selection Problem (Hidden Information)

#### 2.1 The Optimal Screening Menu
Here, the Agent has hidden information about their "type" *before* signing a contract. The Principal's challenge is to design a menu of contracts that induces agents to sort themselves correctly. The canonical example is a monopolist selling a product to consumers with different valuations.

The solution involves a trade-off between **efficiency** and **information rent extraction**. The typical result is:
1.  **Efficiency at the Top:** The high-value type is offered the socially efficient quantity.
2.  **Downward Distortion:** The low-value type is offered an inefficiently low quantity to make their contract less attractive to the high type.
3.  **No Rent at the Bottom:** The low-value type gets zero surplus.
4.  **Information Rent for the High Type:** The high-value type gets a positive surplus, as the Principal must leave them with enough surplus to prevent them from pretending to be a low type.

In [ ]:
sec("Visualizing the Adverse Selection Solution")

def u(q, theta): return theta * np.sqrt(q)
def solve_screening_problem(theta_H=10, theta_L=5, c=1):
    q_fb_H = (theta_H / (2 * c))**2
    q_sb_L = ((2*theta_L - theta_H) / (2 * c))**2 if (2*theta_L > theta_H) else 0
    p_sb_L = u(q_sb_L, theta_L)
    p_sb_H = u(q_fb_H, theta_H) - (u(q_sb_L, theta_H) - p_sb_L)
    return (q_fb_H, p_sb_H), (q_sb_L, p_sb_L)

(qH, pH), (qL, pL) = solve_screening_problem()
note(f"Optimal Menu: High type gets ({qH:.2f} units for ${pH:.2f}). Low type gets ({qL:.2f} units for ${pL:.2f}).")

q_grid = np.linspace(0.01, 100, 200)
plt.figure(figsize=(12,8))
plt.plot(q_grid, u(q_grid, 10), 'b-', label='High Type IC')
plt.plot(q_grid, u(q_grid, 5), 'g-', label='Low Type IC')
plt.plot(q_grid, u(q_grid, 10) - u(qL, 10) + pL, 'b--', label='High Type IC through (qL, pL)')
plt.plot([qH, qL], [pH, pL], 'ro', markersize=12, label='Offered Contracts')
plt.title('Screening: Optimal Menu of Contracts'); plt.xlabel('Quantity (q)'); plt.ylabel('Payment (p)')
plt.legend(); plt.grid(True); plt.ylim(0, 80); plt.show()

#### 2.2 The Revelation Principle
A foundational result in mechanism design is the **Revelation Principle**. It states that any outcome that can be achieved by *any* mechanism (no matter how complex) can also be achieved by a **direct, incentive-compatible mechanism** where agents truthfully report their private information. This dramatically simplifies the search for an optimal mechanism, as we only need to consider the much smaller class of direct, truthful mechanisms.

### 3. Application: Optimal Unemployment Insurance
The design of an optimal unemployment insurance (UI) system is a classic principal-agent problem. The government (Principal) wants to provide insurance to risk-averse workers (Agents) against job loss. However, this creates moral hazard: generous UI benefits may reduce an agent's incentive to search for a new job.

The **Baily-Chetty formula** characterizes the optimal UI replacement rate, $b$, balancing this trade-off:
$$ \frac{u'(c_e) - u'(c_u)}{u'(c_e)} = \frac{\epsilon_s}{\eta_b} $$
Where $c_e$ and $c_u$ are consumption when employed and unemployed, $\epsilon_s$ is the elasticity of unemployment duration with respect to benefits, and $\eta_b$ is the mechanical effect of benefits on the government budget. The formula says that the marginal benefit of insurance (LHS) should equal the marginal cost of distorted incentives (RHS).

### 4. Chapter Summary
- **Asymmetric Information** is a core source of market friction, leading to problems of **Moral Hazard** (hidden action) and **Adverse Selection** (hidden information).
- **Moral Hazard** contracts must balance providing incentives against imposing costly risk on risk-averse agents. In dynamic settings, continuation values can also be used as an incentive device.
- **Adverse Selection** problems are solved by designing a menu of contracts that induces agents to self-select based on their private information. This typically involves distorting the allocation for low types to reduce the information rent paid to high types.
- **The Revelation Principle** is a powerful theoretical tool that simplifies the design of optimal mechanisms.
- **Applications** of principal-agent theory are vast, covering everything from executive compensation and insurance design to optimal taxation and regulation.

### 5. Exercises

1.  **The Informativeness Principle:** A manager's performance is measured by both accounting profits (noisy) and the firm's stock price (very noisy). How should the manager's bonus be structured? Should it depend more on profits or the stock price? Why?

2.  **Limited Liability:** Re-solve the moral hazard model with a **limited liability** constraint that wages cannot be negative ($w_H, w_L \ge 0$). How does this affect the optimal contract, especially when the agent is highly risk-averse?

3.  **Adverse Selection in Insurance:** A health insurance company wants to offer policies to high-risk and low-risk customers. Use the logic of the screening model to describe the menu of contracts (premium and coverage level) the insurer should offer. Which type gets full insurance? Which type gets a contract with a high deductible?

4.  **Dynamic Moral Hazard:** Consider a two-period model where the agent can be fired after the first period. If the firm commits to firing the agent if output is low, but retaining them if output is high, how does this threat affect the wage spread ($w_H - w_L$) needed to incentivize effort in the first period compared to the static model?

5.  **Optimal UI:** The Baily-Chetty formula shows that the optimal UI replacement rate depends on the elasticity of unemployment duration with respect to benefits, $\epsilon_s$. If new empirical work finds this elasticity is larger than previously thought, should the optimal UI system become more or less generous? Explain.